# **Chapter 27: Mobile Testing Types**

---

## **27.1 Introduction to Mobile Testing Types**

Mobile applications operate in a uniquely challenging ecosystem compared to traditional software. While web applications run in controlled browser environments and desktop applications enjoy abundant resources, mobile apps must contend with hardware fragmentation, variable network conditions, resource constraints, and constant interruptions.

**Why Mobile Testing Requires Specialized Approaches:**

1. **Context-Aware Computing:** Mobile apps must adapt to location, orientation, network availability, and battery state in real-time
2. **Hardware Integration:** Direct interaction with sensors, cameras, biometric scanners, and proprietary chipsets varies by device
3. **Platform Governance:** Apple and Google enforce strict behavioral guidelines that can result in app rejection if violated
4. **User Expectations:** Mobile users expect sub-second response times and instant feedback on touch interactions
5. **Lifecycle Complexity:** Apps transition between foreground, background, and terminated states with data preservation requirements

This chapter provides a comprehensive framework for testing mobile applications across all dimensions, ensuring your app not only functions correctly but delivers a premium user experience that meets platform-specific standards.

---

## **27.2 Functional Testing**

Functional testing validates that the application performs according to specifications. In mobile contexts, this extends beyond feature verification to include mobile-specific workflows.

### **27.2.1 Core User Flow Testing**

**Critical Mobile User Journeys:**

```python
class MobileFunctionalTesting:
    """
    Testing core user flows with mobile-specific considerations
    """
    
    def onboarding_flow_test(self, driver):
        """
        Test first-time user experience (FTUE)
        """
        # Splash screen verification
        splash = driver.find_element(AppiumBy.ID, "splash_screen")
        assert splash.is_displayed()
        
        # Tutorial screens (if applicable)
        try:
            tutorial_next = driver.find_element(AppiumBy.ID, "tutorial_next")
            for i in range(3):  # 3-page tutorial
                tutorial_next.click()
                time.sleep(0.5)  # Wait for animation
        except NoSuchElementException:
            pass  # No tutorial
        
        # Permission requests (platform-specific timing)
        if driver.capabilities['platformName'] == 'Android':
            allow_btn = driver.find_element(AppiumBy.ID, "com.android.packageinstaller:id/permission_allow_button")
            allow_btn.click()
        
        # Registration/Login gate
        login_screen = driver.find_element(AppiumBy.ID, "login_screen")
        assert login_screen.is_displayed()
    
    def transaction_flow_with_interruptions(self, driver):
        """
        Test critical transaction (e.g., purchase, booking) handling interruptions
        """
        # Navigate to checkout
        driver.find_element(AppiumBy.ID, "add_to_cart").click()
        driver.find_element(AppiumBy.ID, "checkout").click()
        
        # Fill payment form
        driver.find_element(AppiumBy.ID, "card_number").send_keys("4111111111111111")
        
        # Simulate incoming call (interruption)
        if driver.capabilities['platformName'] == 'Android':
            # Use ADB to simulate call
            os.system("adb shell am start -a android.intent.action.CALL -d tel:555-1234")
            time.sleep(2)
            os.system("adb shell input keyevent 6")  # End call
        
        # Verify form data persisted
        card_field = driver.find_element(AppiumBy.ID, "card_number")
        assert card_field.text == "4111111111111111"
        
        # Complete transaction
        driver.find_element(AppiumBy.ID, "pay_now").click()
        
        # Verify success with receipt
        success = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "success_message"))
        )
        assert "Confirmed" in success.text
    
    def offline_functionality_test(self, driver):
        """
        Test app behavior when offline (airplane mode)
        """
        # Enable airplane mode
        if driver.capabilities['platformName'] == 'Android':
            driver.toggle_airplane_mode()
        
        # Attempt action requiring network
        driver.find_element(AppiumBy.ID, "refresh_data").click()
        
        # Verify offline indicator
        offline_banner = driver.find_element(AppiumBy.ID, "offline_indicator")
        assert offline_banner.is_displayed()
        
        # Verify local data display
        cached_data = driver.find_element(AppiumBy.ID, "cached_list")
        assert cached_data.is_displayed()
        
        # Disable airplane mode
        if driver.capabilities['platformName'] == 'Android':
            driver.toggle_airplane_mode()
        
        # Verify sync occurs
        sync_indicator = driver.find_element(AppiumBy.ID, "syncing_indicator")
        assert sync_indicator.is_displayed()
```

### **27.2.2 Business Logic Validation**

Mobile apps often have complex state management due to lifecycle events:

```python
def test_state_management_across_lifecycle(self, driver):
    """
    Test that business logic state persists correctly
    """
    # Perform multi-step form
    driver.find_element(AppiumBy.ID, "step1_field").send_keys("Data 1")
    driver.find_element(AppiumBy.ID, "next_button").click()
    
    driver.find_element(AppiumBy.ID, "step2_field").send_keys("Data 2")
    
    # Background the app (Home button)
    driver.background_app(5)  # 5 seconds in background
    
    # App returns to foreground
    driver.activate_app("com.example.app")  # iOS bundle ID or Android package
    
    # Verify state maintained
    step2_value = driver.find_element(AppiumBy.ID, "step2_field").text
    assert step2_value == "Data 2"
```

---

## **27.3 UI/UX Testing**

Mobile UI/UX testing validates adherence to platform design guidelines (Material Design for Android, Human Interface Guidelines for iOS) and ensures touch-friendly interfaces.

### **27.3.1 Touch Target and Gesture Testing**

**Minimum Touch Target Compliance:**
- iOS: 44×44 points minimum
- Android: 48×48dp minimum (Material Design)
- WCAG: 44×44 CSS pixels for accessibility

```python
class TouchTargetTesting:
    """
    Validate touch target sizes meet platform guidelines
    """
    
    def verify_touch_target_size(self, driver, element_id):
        """
        Verify element meets minimum touch target size
        """
        element = driver.find_element(AppiumBy.ID, element_id)
        size = element.size
        
        width_px = size['width']
        height_px = size['height']
        
        # Convert to dp (density-independent pixels)
        # Get screen density
        density = driver.execute_script('mobile: getDeviceDensity')['density']
        
        width_dp = width_px / density
        height_dp = height_px / density
        
        platform = driver.capabilities['platformName']
        
        if platform == 'Android':
            assert width_dp >= 48, f"Width {width_dp}dp < 48dp minimum"
            assert height_dp >= 48, f"Height {height_dp}dp < 48dp minimum"
        else:  # iOS
            assert width_px >= 44, f"Width {width_px}pt < 44pt minimum"
            assert height_px >= 44, f"Height {height_px}pt < 44pt minimum"
        
        return True
    
    def test_gesture_recognition(self, driver):
        """
        Test common mobile gestures
        """
        # Swipe gestures
        actions = ActionChains(driver)
        
        # Swipe left (delete item)
        list_item = driver.find_element(AppiumBy.ID, "list_item_1")
        actions.click_and_hold(list_item).move_by_offset(-300, 0).release().perform()
        
        # Verify delete confirmation appeared
        confirm_delete = driver.find_element(AppiumBy.ID, "delete_confirmation")
        assert confirm_delete.is_displayed()
        
        # Pinch to zoom (on map/image)
        map_view = driver.find_element(AppiumBy.ID, "map_view")
        
        # Two-finger gesture simulation
        finger1 = PointerInput(interaction.POINTER_TOUCH, "finger1")
        finger2 = PointerInput(interaction.POINTER_TOUCH, "finger2")
        
        # Start positions (30px apart)
        actions1 = ActionBuilder(driver, mouse=finger1)
        actions1.pointer_action.move_to_location(300, 400).pointer_down()
        
        actions2 = ActionBuilder(driver, mouse=finger2)
        actions2.pointer_action.move_to_location(330, 400).pointer_down()
        
        # Move apart (zoom in)
        actions1.pointer_action.move_to_location(200, 400)
        actions2.pointer_action.move_to_location(430, 400)
        
        actions1.pointer_action.pointer_up()
        actions2.pointer_action.pointer_up()
        
        actions1.perform()
        actions2.perform()
        
        # Verify zoom level changed
        zoom_level = driver.find_element(AppiumBy.ID, "zoom_indicator").text
        assert float(zoom_level) > 1.0
    
    def test_haptic_feedback(self, driver):
        """
        Verify haptic feedback triggers (iOS/Android 10+)
        """
        # Perform action that should trigger haptic
        driver.find_element(AppiumBy.ID, "success_action").click()
        
        # Haptic feedback is hardware-level; verify through logs or accessibility
        # On iOS, can verify through system logs
        if driver.capabilities['platformName'] == 'iOS':
            logs = driver.get_log('syslog')
            haptic_entries = [log for log in logs if 'haptic' in log['message'].lower()]
            # This is indirect verification; primarily manual testing
```

### **27.3.2 Platform Guidelines Compliance**

```python
class PlatformGuidelines:
    """
    Verify adherence to iOS HIG and Material Design
    """
    
    def test_ios_human_interface_guidelines(self, driver):
        """
        iOS-specific UI compliance checks
        """
        checks = []
        
        # 1. Dynamic Type support (text size accessibility)
        # Verify labels adjust to larger text sizes
        driver.execute_script('mobile: setAppearance', {'style': 'dark'})
        # Check dark mode adaptation
        
        # 2. Safe Area compliance (notch/island handling)
        # Take screenshot and verify critical UI not obscured
        screenshot = driver.get_screenshot_as_base64()
        # Automated analysis would check for elements in safe area
        
        # 3. Back navigation (iOS uses swipe from left edge or back button)
        if driver.find_element(AppiumBy.ID, "back_button").is_displayed():
            checks.append("Back button present")
        
        # 4. Tab Bar height (iOS standard is 49pt)
        tab_bar = driver.find_element(AppiumBy.CLASS_NAME, "XCUIElementTypeTabBar")
        height = tab_bar.size['height']
        assert height >= 49, "Tab bar too short for iOS"
        
        return checks
    
    def test_material_design_compliance(self, driver):
        """
        Android Material Design compliance
        """
        # 1. Elevation/shadows (API 21+)
        # Check elevation attribute on cards
        card = driver.find_element(AppiumBy.ID, "card_view")
        elevation = card.get_attribute("elevation")
        assert elevation is not None
        
        # 2. Ripple effects on touch
        # Visual verification primarily, but can check state_pressed
        
        # 3. Navigation drawer behavior (if applicable)
        # Swipe from left edge should open drawer
        actions = ActionChains(driver)
        actions.move_to_location(50, 600).click_and_hold().move_by_offset(300, 0).release().perform()
        
        drawer = driver.find_element(AppiumBy.ID, "navigation_drawer")
        assert drawer.is_displayed()
        
        # 4. Floating Action Button (FAB) positioning
        fab = driver.find_element(AppiumBy.ID, "fab_button")
        location = fab.location
        size = driver.get_window_size()
        
        # FAB typically 16-24dp from bottom-right
        assert location['x'] > size['width'] * 0.7  # Right side
        assert location['y'] > size['height'] * 0.8  # Bottom
```

---

## **27.4 Cross-Platform Testing**

Cross-platform testing ensures consistent functionality and user experience across iOS and Android, accounting for platform conventions while maintaining brand consistency.

### **27.4.1 Consistency Validation Framework**

```python
class CrossPlatformTesting:
    """
    Validate feature parity and consistency across platforms
    """
    
    def test_feature_parity(self, ios_driver, android_driver):
        """
        Verify both platforms have equivalent features
        """
        # Test data
        test_user = "parity_test_user"
        test_pass = "password123"
        
        # iOS Login
        self.login(ios_driver, test_user, test_pass)
        ios_features = self.extract_available_features(ios_driver)
        
        # Android Login
        self.login(android_driver, test_user, test_pass)
        android_features = self.extract_available_features(android_driver)
        
        # Compare feature sets (allowing for platform-specific differences)
        core_features_ios = set(f for f in ios_features if not f.endswith('_ios'))
        core_features_android = set(f for f in android_features if not f.endswith('_android'))
        
        missing_in_android = core_features_ios - core_features_android
        missing_in_ios = core_features_android - core_features_ios
        
        assert not missing_in_android, f"Features missing in Android: {missing_in_android}"
        assert not missing_in_ios, f"Features missing in iOS: {missing_in_ios}"
    
    def test_data_consistency(self, ios_driver, android_driver):
        """
        Verify data sync and format consistency across platforms
        """
        # Create data on iOS
        ios_driver.find_element(AppiumBy.ID, "create_note").click()
        ios_driver.find_element(AppiumBy.ID, "note_text").send_keys("Cross-platform test")
        ios_driver.find_element(AppiumBy.ID, "save").click()
        
        # Allow sync time
        time.sleep(5)
        
        # Verify on Android
        android_driver.find_element(AppiumBy.ID, "refresh").click()
        time.sleep(2)
        
        notes = android_driver.find_elements(AppiumBy.ID, "note_item")
        note_texts = [n.text for n in notes]
        
        assert "Cross-platform test" in note_texts, "Data not synced correctly"
        
        # Verify date formats are platform-appropriate
        # iOS: "Jan 15, 2026" vs Android: "15 Jan 2026"
        date_element_ios = ios_driver.find_element(AppiumBy.ID, "note_date")
        date_element_android = android_driver.find_element(AppiumBy.ID, "note_date")
        
        # Verify formats match platform conventions
        if "iOS" in ios_driver.capabilities['platformName']:
            assert "," in date_element_ios.text  # US format
```

### **27.4.2 Platform-Specific Adaptation Testing**

```python
def test_platform_adaptations(self, driver, platform):
    """
    Verify platform-specific UI adaptations are correct
    """
    if platform == "Android":
        # Android typically uses top app bar with hamburger menu
        try:
            hamburger = driver.find_element(AppiumBy.ACCESSIBILITY_ID, "Open navigation drawer")
            assert hamburger.is_displayed()
        except NoSuchElementException:
            # Or bottom navigation
            bottom_nav = driver.find_element(AppiumBy.ID, "bottom_navigation")
            assert bottom_nav.is_displayed()
            
        # Android back button behavior
        driver.press_keycode(4)  # Android back
        # Verify appropriate navigation (up vs back handled correctly)
        
    elif platform == "iOS":
        # iOS typically uses bottom tab bar
        tab_bar = driver.find_element(AppiumBy.CLASS_NAME, "XCUIElementTypeTabBar")
        assert tab_bar.is_displayed()
        
        # iOS swipe-to-go-back gesture
        actions = ActionChains(driver)
        actions.move_to_location(50, 400).click_and_hold().move_by_offset(100, 0).release().perform()
        # Verify navigation occurred
```

---

## **27.5 Cross-Browser Testing (Hybrid/PWA)**

For hybrid apps and Progressive Web Apps (PWAs), testing across mobile browsers is essential as rendering engines and capabilities vary.

### **27.5.1 Mobile Browser Matrix**

```python
class MobileCrossBrowserTesting:
    """
    Testing hybrid apps across mobile browsers
    """
    
    BROWSER_MATRIX = {
        "Android": [
            {"browser": "Chrome", "versions": ["120", "119", "118"], "engine": "Blink"},
            {"browser": "Samsung Internet", "versions": ["23", "22"], "engine": "Blink"},
            {"browser": "Firefox", "versions": ["121", "120"], "engine": "Gecko"}
        ],
        "iOS": [
            {"browser": "Safari", "versions": ["17", "16"], "engine": "WebKit"},
            {"browser": "Chrome", "versions": ["120"], "engine": "WebKit"},  # iOS Chrome uses WebKit
            {"browser": "Firefox", "versions": ["120"], "engine": "WebKit"}  # iOS Firefox uses WebKit
        ]
    }
    
    def test_webview_rendering_consistency(self, driver, browser):
        """
        Verify WebView renders consistently across browsers
        """
        # Navigate to WebView screen
        driver.find_element(AppiumBy.ID, "web_content").click()
        
        # Switch to WebView context
        webview_context = [c for c in driver.contexts if 'WEBVIEW' in c][0]
        driver.switch_to.context(webview_context)
        
        # Execute JavaScript to check rendering
        render_info = driver.execute_script("""
            return {
                'userAgent': navigator.userAgent,
                'viewport': {'width': window.innerWidth, 'height': window.innerHeight},
                'flexboxSupport': CSS.supports('display', 'flex'),
                'gridSupport': CSS.supports('display', 'grid')
            };
        """)
        
        # Verify critical CSS features supported
        assert render_info['flexboxSupport'], "Flexbox not supported"
        
        # Check layout consistency
        driver.find_element(By.ID, "main_container").is_displayed()
        
        # Return to native context
        driver.switch_to.context('NATIVE_APP')
```

### **27.5.2 PWA Browser Capabilities**

```python
def test_pwa_capabilities_by_browser(self, driver, browser_name):
    """
    Test PWA feature support varies by browser
    """
    if browser_name == "Safari":
        # iOS Safari limitations
        # No push notifications (until recently), limited background sync
        capabilities = {
            'service_worker': True,
            'push_notifications': False,  # Check iOS version
            'background_sync': False,
            'add_to_home': True  # Manual only
        }
    elif browser_name == "Chrome":
        capabilities = {
            'service_worker': True,
            'push_notifications': True,
            'background_sync': True,
            'add_to_home': True
        }
    
    # Test Service Worker registration
    driver.get("https://example.com/pwa")
    
    sw_registered = driver.execute_script("""
        return navigator.serviceWorker !== undefined;
    """)
    
    assert sw_registered, "Service Worker not registered"
```

---

## **27.6 Responsive Design Testing**

Mobile devices span from 4-inch phones to 13-inch tablets, with foldable devices adding complexity. Responsive testing ensures UI adapts gracefully.

### **27.6.1 Screen Size and Density Testing**

```python
class ResponsiveTesting:
    """
    Testing responsive layouts across device sizes
    """
    
    def test_layout_adaptation(self, driver):
        """
        Verify layout changes appropriately for screen size
        """
        # Get device metrics
        metrics = driver.execute_script('mobile: getDeviceDensity')
        width_px = driver.get_window_size()['width']
        height_px = driver.get_window_size()['height']
        
        # Calculate logical width (dp for Android, points for iOS)
        platform = driver.capabilities['platformName']
        if platform == 'Android':
            logical_width = width_px / metrics['density']
        else:
            logical_width = width_px / metrics['scale']  # iOS uses scale factor
        
        if logical_width < 360:  # Small phone
            # Should show single column layout
            layout = driver.find_element(AppiumBy.ID, "single_column_layout")
            assert layout.is_displayed()
        elif logical_width < 600:  # Normal phone
            layout = driver.find_element(AppiumBy.ID, "phone_layout")
            assert layout.is_displayed()
        else:  # Tablet or large phone
            layout = driver.find_element(AppiumBy.ID, "tablet_layout")
            assert layout.is_displayed()
    
    def test_orientation_changes(self, driver):
        """
        Test rotation between portrait and landscape
        """
        # Portrait state
        driver.orientation = 'PORTRAIT'
        portrait_element = driver.find_element(AppiumBy.ID, "portrait_only_element")
        assert portrait_element.is_displayed()
        
        # Record state
        form_data = driver.find_element(AppiumBy.ID, "text_field").text
        
        # Rotate to landscape
        driver.orientation = 'LANDSCAPE'
        time.sleep(1)  # Wait for rotation animation
        
        # Verify UI adapted
        landscape_element = driver.find_element(AppiumBy.ID, "landscape_optimized_layout")
        assert landscape_element.is_displayed()
        
        # Verify data persisted
        form_data_after = driver.find_element(AppiumBy.ID, "text_field").text
        assert form_data == form_data_after, "Data lost on rotation"
        
        # Rotate back
        driver.orientation = 'PORTRAIT'
    
    def test_foldable_devices(self, driver):
        """
        Test foldable device states (folded/unfolded)
        """
        # Check if running on foldable emulator/device
        folding_state = driver.execute_script('mobile: shell', {
            'command': 'wm size'
        })  # Android command to check display size
        
        # Test folded (outer screen)
        # Should show compact UI
        compact_mode = driver.find_element(AppiumBy.ID, "compact_mode_indicator")
        
        # Simulate unfold (if supported by test environment)
        # In real testing, this requires physical device or specific emulator features
        
        # Verify expanded layout
        expanded_mode = driver.find_element(AppiumBy.ID, "expanded_mode_indicator")
        assert expanded_mode.is_displayed() or compact_mode.is_displayed()
```

---

## **27.7 Compatibility Testing**

Compatibility testing ensures the app works across OS versions, manufacturer customizations, and hardware variations.

### **27.7.1 OS Version Compatibility**

```python
class OSCompatibilityTesting:
    """
    Testing across Android API levels and iOS versions
    """
    
    def test_android_api_compatibility(self, driver, api_level):
        """
        Test behavior differences across Android versions
        """
        if api_level >= 33:  # Android 13+
            # Themed app icons support
            themed_icon = driver.find_element(AppiumBy.ID, "launcher_icon")
            # Verify themed icon exists
            
            # Per-app language preferences
            language_settings = driver.find_element(AppiumBy.ID, "language_settings")
            assert language_settings.is_displayed()
            
        elif api_level >= 31:  # Android 12+
            # Approximate location permission
            location_permission = driver.find_element(AppiumBy.ID, "approximate_location")
            
        elif api_level >= 29:  # Android 10+
            # Scoped storage restrictions
            # Verify file access uses Storage Access Framework
            try:
                driver.find_element(AppiumBy.ID, "legacy_file_access")
                assert False, "Should not use legacy file access on API 29+"
            except NoSuchElementException:
                pass  # Correct - using modern storage
    
    def test_ios_version_compatibility(self, driver, ios_version):
        """
        Test iOS version-specific features
        """
        if ios_version >= 17.0:
            # Standby mode widgets (if applicable)
            pass
            
        if ios_version >= 16.0:
            # Lock Screen widgets
            pass
            
        if ios_version >= 15.0:
            # Focus modes integration
            focus_status = driver.execute_script('mobile: activateApp', {
                'bundleId': 'com.apple.focus'
            })
```

### **27.7.2 Manufacturer Customization Testing**

```python
def test_manufacturer_customizations(self, driver, manufacturer):
    """
    Test behavior on different OEM Android skins
    """
    if manufacturer == "Samsung":
        # One UI specific behaviors
        # Edge panels might interfere with swipe gestures
        try:
            # Test swipe from edge
            actions = ActionChains(driver)
            actions.move_to_location(0, 500).click_and_hold().move_by_offset(100, 0).release().perform()
        except:
            pass  # May trigger edge panel
        
        # Battery optimization might affect background tasks
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys deviceidle'
        })
        
    elif manufacturer == "Xiaomi":
        # MIUI aggressive battery saving
        # Verify app whitelisted for background operation
        pass
        
    elif manufacturer == "Huawei":
        # HMS (Huawei Mobile Services) vs GMS
        # Verify graceful degradation when Google Play Services unavailable
        try:
            gms_check = driver.find_element(AppiumBy.ID, "gms_feature")
            # If present, should handle absence gracefully
        except:
            pass
```

---

## **27.8 Performance Testing**

Mobile performance directly impacts user retention. Studies show 53% of users abandon apps that take longer than 3 seconds to load.

### **27.8.1 Launch Time Testing**

```python
class PerformanceTesting:
    """
    Mobile-specific performance metrics
    """
    
    def test_cold_start_time(self, driver):
        """
        Measure cold start duration (app not in memory)
        """
        # Terminate app completely
        driver.terminate_app("com.example.app")
        
        start_time = time.time()
        driver.activate_app("com.example.app")
        
        # Wait for first meaningful paint (home screen loaded)
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((AppiumBy.ID, "home_screen_loaded"))
        )
        
        end_time = time.time()
        cold_start_duration = end_time - start_time
        
        # Industry standard: Cold start < 3 seconds
        assert cold_start_duration < 3.0, f"Cold start too slow: {cold_start_duration}s"
        
        return cold_start_duration
    
    def test_warm_start_time(self, driver):
        """
        Measure warm start (app in background, not terminated)
        """
        # Background app
        driver.background_app(5)
        
        start_time = time.time()
        driver.activate_app("com.example.app")
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((AppiumBy.ID, "home_screen"))
        )
        
        warm_start = time.time() - start_time
        assert warm_start < 1.5, f"Warm start too slow: {warm_start}s"
    
    def test_frame_rate(self, driver):
        """
        Test UI rendering performance (FPS)
        """
        # Start FPS monitoring
        driver.execute_script('mobile: startPerfRecord', {
            'timeout': 30000,
            'profileName': 'systrace'  # or 'simpleperf'
        })
        
        # Perform scroll gesture (common jank source)
        for _ in range(5):
            driver.swipe(500, 1000, 500, 200, 1000)  # Slow scroll
        
        # Stop and get results
        result = driver.execute_script('mobile: stopPerfRecord')
        
        # Parse FPS from result (platform-specific format)
        # Target: 60 FPS (16.67ms per frame)
        # Acceptable: > 55 FPS average, < 5% frames > 32ms (jank)
```

### **27.8.2 Memory and Battery Testing**

```python
def test_memory_leaks(self, driver):
    """
    Detect memory leaks through repeated operations
    """
    # Get initial memory usage
    initial_memory = driver.execute_script('mobile: shell', {
        'command': 'dumpsys meminfo com.example.app | grep "TOTAL"'
    })
    
    # Perform action 20 times (e.g., open/close screens)
    for i in range(20):
        driver.find_element(AppiumBy.ID, "open_details").click()
        time.sleep(0.5)
        driver.find_element(AppiumBy.ID, "back").click()
        time.sleep(0.5)
    
    # Force garbage collection (if possible)
    driver.execute_script('mobile: shell', {
        'command': 'am send-trim-memory com.example.app RUNNING_CRITICAL'
    })
    
    # Get final memory
    final_memory = driver.execute_script('mobile: shell', {
        'command': 'dumpsys meminfo com.example.app | grep "TOTAL"'
    })
    
    # Parse values and compare
    # Memory should return to near initial levels
    # If significantly higher, indicates leak

def test_battery_drain(self, driver):
    """
    Measure battery consumption during typical usage
    """
    # Reset battery stats (requires root or debug build)
    driver.execute_script('mobile: shell', {
        'command': 'dumpsys batterystats --reset'
    })
    
    # Perform 30 minutes of typical usage simulation
    end_time = time.time() + 1800
    while time.time() < end_time:
        driver.find_element(AppiumBy.ID, "scroll_feed").swipe(500, 1000, 500, 500)
        time.sleep(2)
    
    # Get battery stats
    stats = driver.execute_script('mobile: shell', {
        'command': 'dumpsys batterystats com.example.app | grep "Computed drain"'
    })
    
    # Parse mAh consumed
    # Acceptable: < 5% per hour for background, < 15% for active use
```

---

## **27.9 Security Testing**

Mobile apps handle sensitive data and face unique threats (jailbroken devices, intercepted communications, reverse engineering).

### **27.9.1 OWASP Mobile Top 10 Validation**

```python
class MobileSecurityTesting:
    """
    Automated security checks for mobile apps
    """
    
    def test_ssl_pinning(self, driver):
        """
        Verify certificate pinning prevents MITM attacks
        """
        # Attempt to proxy through Charles/Fiddler with custom cert
        # If pinning works, app should fail to connect or show error
        
        # Trigger network request
        driver.find_element(AppiumBy.ID, "fetch_data").click()
        
        # Check for SSL error message (if pinning active)
        try:
            error_msg = driver.find_element(AppiumBy.ID, "ssl_error")
            assert "certificate" in error_msg.text.lower()
            print("SSL Pinning active - Good")
        except NoSuchElementException:
            # If request succeeds, pinning might not be implemented
            # Verify through logs or proxy interception status
            pass
    
    def test_root_detection(self, driver):
        """
        Verify app detects rooted/jailbroken devices
        """
        # Check for root detection dialog
        try:
            root_warning = driver.find_element(AppiumBy.ID, "root_warning")
            assert root_warning.is_displayed()
            print("Root detection working")
        except NoSuchElementException:
            # May be running on non-rooted device or detection bypassed
            pass
    
    def test_data_storage_security(self, driver):
        """
        Verify sensitive data not stored insecurely
        """
        # Check SharedPreferences/Keychain for sensitive data
        # This requires access to app sandbox (debug build or rooted)
        
        prefs = driver.execute_script('mobile: shell', {
            'command': 'cat /data/data/com.example.app/shared_prefs/app_prefs.xml'
        })
        
        # Verify no plaintext passwords
        assert 'password' not in prefs.lower() or 'enc_' in prefs, "Unencrypted password found"
        
        # Verify SQLite database encryption (if applicable)
        # Check for SQLCipher or similar indicators
    
    def test_clipboard_security(self, driver):
        """
        Verify sensitive fields don't allow copy/paste or clear clipboard
        """
        # Enter password
        pwd_field = driver.find_element(AppiumBy.ID, "password_field")
        pwd_field.send_keys("secret123")
        
        # Long press to see if copy option appears
        actions = ActionChains(driver)
        actions.click_and_hold(pwd_field).pause(2).release().perform()
        
        # Check for copy/paste menu (should not appear for secure fields)
        try:
            copy_menu = driver.find_element(AppiumBy.ID, "android:id/copy")
            assert False, "Copy allowed on password field"
        except NoSuchElementException:
            pass  # Good - copy not available
```

### **27.9.2 Runtime Application Security Protection (RASP)**

```python
def test_anti_tampering(self, driver):
    """
    Verify app integrity checks
    """
    # Try to modify app code (requires repackaging)
    # Or check for debugger detection
    
    # Attach debugger and verify app detects or crashes
    # (This is more of a manual/security audit test)
    
    # Check for debug mode flags
    debug_mode = driver.execute_script('mobile: shell', {
        'command': 'getprop ro.debuggable'
    })
    
    if debug_mode == '1':
        # App should detect debug environment
        warning = driver.find_element(AppiumBy.ID, "debug_warning")
        assert warning.is_displayed()
```

---

## **27.10 Installation and Update Testing**

App distribution mechanics require specific testing for OTA (Over-The-Air) updates, migrations, and store compliance.

### **27.10.1 Installation Scenarios**

```python
class InstallationTesting:
    """
    Testing app installation and updates
    """
    
    def test_fresh_install(self, driver):
        """
        Test clean installation flow
        """
        # Uninstall if present
        driver.remove_app("com.example.app")
        
        # Install fresh
        driver.install_app("/path/to/app.apk")
        driver.activate_app("com.example.app")
        
        # Verify first launch tutorial/onboarding shown
        onboarding = driver.find_element(AppiumBy.ID, "onboarding_screen")
        assert onboarding.is_displayed()
        
        # Verify no crash on first launch
        assert driver.find_element(AppiumBy.ID, "app_root").is_displayed()
    
    def test_update_scenarios(self, driver):
        """
        Test app update from previous versions
        """
        # Install old version
        driver.install_app("/path/to/app_v1.0.apk")
        driver.activate_app("com.example.app")
        
        # Create data in old version
        driver.find_element(AppiumBy.ID, "create_profile").click()
        driver.find_element(AppiumBy.ID, "name").send_keys("Test User")
        driver.find_element(AppiumBy.ID, "save").click()
        
        # Update to new version (OTA)
        driver.install_app("/path/to/app_v2.0.apk")
        driver.activate_app("com.example.app")
        
        # Verify data migrated correctly
        profile_name = driver.find_element(AppiumBy.ID, "profile_name")
        assert profile_name.text == "Test User", "Data lost during update"
        
        # Verify new features available
        new_feature = driver.find_element(AppiumBy.ID, "new_v2_feature")
        assert new_feature.is_displayed()
    
    def test_delta_update(self, driver):
        """
        Test that delta updates (Play Store/App Store) don't corrupt app
        """
        # This requires specific store testing tracks
        # Upload v1 to internal testing
        # Upload v2 (should be delta)
        # Update and verify functionality
        
        pass  # Typically done via store console testing
```

### **27.10.2 Storage and Permission Handling**

```python
def test_storage_migration(self, driver):
    """
    Test Scoped Storage migration (Android 10+)
    """
    # On Android 9, create file in external storage
    # Update to Android 10+ target SDK
    # Verify app still accesses files correctly through SAF (Storage Access Framework)
    # Or verify migration to app-specific directories
    
    if driver.capabilities['platformVersion'] >= 10:
        # Try to access old external storage paths
        result = driver.execute_script('mobile: shell', {
            'command': 'ls /sdcard/legacy_app_folder'
        })
        # Should fail or be empty (if properly migrated)
```

---

## **27.11 Interruption Testing**

Mobile apps must handle system interruptions gracefully without data loss or crashes.

### **27.11.1 System Event Handling**

```python
class InterruptionTesting:
    """
    Testing system interruptions
    """
    
    def test_incoming_call(self, driver):
        """
        Handle incoming phone call
        """
        # Start recording audio or video (if app supports it)
        driver.find_element(AppiumBy.ID, "record_video").click()
        
        # Simulate incoming call (ADB for Android)
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'am start -a android.intent.action.CALL -d tel:5551234567'
            })
            time.sleep(2)
            
            # End call
            driver.execute_script('mobile: shell', {
                'command': 'input keyevent 6'  # End call key
            })
        
        # Verify app handled interruption
        # Video should pause or save
        status = driver.find_element(AppiumBy.ID, "recording_status")
        assert "Paused" in status.text or "Saved" in status.text
    
    def test_low_battery(self, driver):
        """
        Test behavior when battery low
        """
        # Simulate low battery (requires emulator or rooted device)
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys battery set level 15'
        })
        
        # Trigger low battery mode
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys battery set status 3'  # Discharging
        })
        
        # Verify app reduces background activity
        # Or shows low battery warning if applicable
        
        # Reset
        driver.execute_script('mobile: shell', {
            'command': 'dumpsys battery reset'
        })
    
    def test_permission_dialog_interruption(self, driver):
        """
        Test handling permission dialogs during user flow
        """
        # Start form filling
        driver.find_element(AppiumBy.ID, "name").send_keys("Test")
        
        # Trigger permission request (camera, location, etc.)
        driver.find_element(AppiumBy.ID, "take_photo").click()
        
        # Handle permission dialog
        try:
            allow_btn = driver.find_element(AppiumBy.ID, "com.android.packageinstaller:id/permission_allow_button")
            allow_btn.click()
        except:
            pass
        
        # Verify form data persisted
        name_field = driver.find_element(AppiumBy.ID, "name")
        assert name_field.text == "Test", "Form data lost during permission dialog"
    
    def test_time_change(self, driver):
        """
        Test app behavior when system time changes
        """
        # Set specific time
        driver.execute_script('mobile: shell', {
            'command': 'date 010100002026.00'  # Set to Jan 1, 2026
        })
        
        # Test time-sensitive features (timers, scheduled tasks)
        # Verify app handles time jump gracefully
        
        # Reset time (requires root or specific permissions)
        driver.execute_script('mobile: shell', {
            'command': 'date $(date +%m%d%H%M%Y.%S)'  # Rough reset
        })
```

---

## **27.12 Network Testing**

Network testing ensures robust behavior across connectivity spectrums from 5G to offline.

### **27.12.1 Connectivity State Testing**

```python
class NetworkTesting:
    """
    Network condition testing
    """
    
    def test_offline_mode(self, driver):
        """
        Verify app works offline (if applicable)
        """
        # Enable airplane mode
        driver.toggle_airplane_mode()
        
        # Try to load content
        driver.find_element(AppiumBy.ID, "refresh").click()
        
        # Should show cached content or offline message
        try:
            offline_msg = driver.find_element(AppiumBy.ID, "offline_message")
            assert offline_msg.is_displayed()
        except:
            content = driver.find_element(AppiumBy.ID, "cached_content")
            assert content.is_displayed()
        
        # Disable airplane mode
        driver.toggle_airplane_mode()
        
        # Verify sync occurs when back online
        sync_indicator = driver.find_element(AppiumBy.ID, "syncing")
        assert sync_indicator.is_displayed()
    
    def test_network_transition(self, driver):
        """
        Test WiFi to Cellular handover
        """
        # Start download or streaming
        driver.find_element(AppiumBy.ID, "stream_video").click()
        
        # Switch network (requires specific setup or proxy)
        # This is typically done via:
        # - Charles Proxy throttling
        # - Network Link Conditioner (iOS)
        # - ADB network commands
        
        # Verify stream continues or gracefully buffers
        buffer_indicator = driver.find_element(AppiumBy.ID, "buffering")
        # Should not crash or show error
    
    def test_slow_network(self, driver):
        """
        Test behavior on 2G/3G speeds
        """
        # Enable network throttling (using proxy or emulator settings)
        # Charles Proxy: Throttle to 56kbps
        
        # Attempt API call
        start_time = time.time()
        driver.find_element(AppiumBy.ID, "fetch_data").click()
        
        # Verify timeout handling
        try:
            error = driver.find_element(AppiumBy.ID, "timeout_error")
            assert error.is_displayed()
        except:
            # Or verify it eventually loads
            WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((AppiumBy.ID, "data_loaded"))
            )
            duration = time.time() - start_time
            print(f"Loaded on slow network in {duration}s")
```

---

## **27.13 Localization and Internationalization Testing**

Mobile apps often serve global markets requiring support for multiple languages, scripts, and cultural conventions.

### **27.13.1 Language and Locale Testing**

```python
class LocalizationTesting:
    """
    Testing internationalization
    """
    
    def test_rtl_layout(self, driver):
        """
        Test Right-to-Left languages (Arabic, Hebrew)
        """
        # Change device language to Arabic
        if driver.capabilities['platformName'] == 'Android':
            driver.execute_script('mobile: shell', {
                'command': 'am start -a android.settings.LOCALE_SETTINGS'
            })
            # Manually change or use app locale changer
        
        # Relaunch app
        driver.activate_app("com.example.app")
        
        # Verify layout mirrored
        # In RTL: Back arrow points right, text aligns right, scroll from left
        
        # Check text direction
        text_element = driver.find_element(AppiumBy.ID, "title_text")
        # Should contain Arabic text
        
        # Verify no truncation or overlap
        assert text_element.is_displayed()
        
        # Test navigation (back should be on right)
        back_button = driver.find_element(AppiumBy.ACCESSIBILITY_ID, "Navigate up")
        location = back_button.location
        screen_width = driver.get_window_size()['width']
        
        # In RTL, back button should be on right side
        assert location['x'] > screen_width / 2
    
    def test_date_time_formats(self, driver):
        """
        Verify localized date/time displays
        """
        locales = [
            {"locale": "en_US", "date_format": "MM/DD/YYYY", "time": "12-hour"},
            {"locale": "en_GB", "date_format": "DD/MM/YYYY", "time": "24-hour"},
            {"locale": "de_DE", "date_format": "DD.MM.YYYY", "separator": "."},
            {"locale": "ja_JP", "date_format": "YYYY/MM/DD", "era": " Reiwa"}
        ]
        
        for locale_config in locales:
            # Set locale
            # Verify dates display correctly
            date_display = driver.find_element(AppiumBy.ID, "event_date")
            displayed_date = date_display.text
            
            # Basic validation (specific checks depend on implementation)
            assert len(displayed_date) > 0
            assert any(sep in displayed_date for sep in ["/", ".", "-"])
    
    def test_text_expansion(self, driver):
        """
        Test UI handles longer text in different languages
        """
        # German text is typically 30% longer than English
        # Test long strings don't break layout
        
        long_german_text = "Donaudampfschifffahrtsgesellschaftskapitänsmütze"
        
        # Enter in text field
        field = driver.find_element(AppiumBy.ID, "description_field")
        field.send_keys(long_german_text)
        
        # Verify field expands or scrolls, doesn't overlap
        assert field.is_displayed()
        
        # Take screenshot for visual validation
        driver.get_screenshot_as_file(f"german_text_{time.time()}.png")
    
    def test_currency_and_number_formats(self, driver):
        """
        Verify currency displays correctly for locale
        """
        # Set locale to Germany
        # Price should show: 1.234,56 € (comma decimal, dot thousands, symbol after)
        
        price = driver.find_element(AppiumBy.ID, "price_display")
        text = price.text
        
        # Contains Euro symbol
        assert '€' in text
        
        # Set locale to US
        # Should show: $1,234.56
        
        price_us = driver.find_element(AppiumBy.ID, "price_display")
        assert '$' in price_us.text
```

---

## **Chapter Summary**

### **Key Takeaways from Chapter 27:**

**Functional Testing:**
- Mobile functional testing must account for lifecycle interruptions (calls, notifications) and state persistence across background/foreground transitions
- Offline functionality is critical; test graceful degradation when network unavailable
- Transaction integrity must be maintained despite interruptions

**UI/UX Testing:**
- Touch targets must meet minimum sizes (48dp Android, 44pt iOS) for accessibility
- Gestures require specific testing: swipe, pinch, long-press, edge cases
- Platform guidelines compliance (Material Design vs. Human Interface Guidelines) ensures app store acceptance and user familiarity

**Cross-Platform & Cross-Browser:**
- Feature parity between iOS and Android must be verified while respecting platform conventions
- Hybrid apps/PWAs require testing across mobile browsers (Chrome, Safari, Samsung Internet) with different capabilities
- Responsive design must handle foldable devices, tablets, and orientation changes

**Compatibility Testing:**
- OS version testing ensures compatibility across API levels (Android) and iOS versions
- Manufacturer customization testing (Samsung One UI, Xiaomi MIUI) prevents crashes on specific OEM skins
- Hardware variation testing covers camera capabilities, sensors, and chipsets

**Performance Testing:**
- Cold start must complete within 3 seconds; warm start within 1.5 seconds
- Frame rate testing targets 60 FPS with <5% jank frames
- Battery drain should not exceed 5% per hour background, 15% active
- Memory leak detection through repeated operation cycles

**Security Testing:**
- SSL pinning prevents MITM attacks; certificate validation required
- Root/jailbreak detection protects against compromised devices
- Data storage encryption: no plaintext sensitive data in SharedPreferences/Keychain
- Clipboard security: password fields must disable copy/paste

**Installation & Updates:**
- OTA updates must preserve user data (database migrations, SharedPreferences)
- Delta updates should not corrupt installation
- Scoped storage migration testing (Android 10+) critical for updates

**Interruption Testing:**
- Incoming calls/SMS must pause/resume media without crashes
- Permission dialogs should not cause data loss in forms
- Low battery mode should trigger graceful feature reduction
- Time changes must not corrupt scheduled tasks

**Network Testing:**
- Offline mode must display cached content or appropriate messaging
- Network transitions (WiFi→Cellular) should not drop connections
- Slow network testing (2G/3G) validates timeout handling and loading states

**Localization:**
- RTL (Right-to-Left) languages require layout mirroring and bidirectional text support
- Date/time formats vary by locale (US: MM/DD/YYYY, EU: DD/MM/YYYY)
- Text expansion (German +30%) must not break layouts
- Currency and number formatting must follow local conventions

---

## **📖 Next Chapter: Chapter 28 - Mobile-Specific Scenarios**

Having covered the comprehensive taxonomy of mobile testing types, **Chapter 28** will dive deep into **specific, complex mobile scenarios** that require specialized testing approaches.

In **Chapter 28**, you'll master:

- **Orientation Changes:** Handling rotation between portrait and landscape, foldable device posture changes, and multi-window mode on tablets
- **Background and Foreground Modes:** App state preservation, background task execution limits (Doze mode, App Standby), and foreground services
- **Push Notifications:** Local vs. remote notifications, notification channels/categories, rich notifications with images/actions, and notification permission handling
- **Location Services:** GPS accuracy testing, background location updates, geofencing boundary testing, and location permission levels (While Using vs Always)
- **Biometric Authentication:** Fingerprint (Touch ID) and Face ID testing, fallback to passcode, and biometric enrollment changes
- **Deep Linking:** Universal Links (iOS) and App Links (Android), custom URL schemes, deferred deep linking, and attribution testing
- **App Permissions:** Runtime permission models, permission rationale dialogs, and handling "Don't ask again" scenarios
- **Memory and Storage:** Low memory killer (LMK) behavior, out-of-memory handling, and external storage (SD card) scenarios
- **Camera and Media:** Image capture quality, camera permissions, photo library access, and media handling with limited storage

**Chapter 28** provides the advanced techniques for handling mobile-specific edge cases that differentiate good apps from great apps in the competitive mobile marketplace.

**Continue to Chapter 28 to master the complex mobile scenarios that ensure your app thrives in real-world conditions!**